In [ ]:
import numpy as np
from GO import GO
from PPI import PPI
from PathManager import PathManager as PM
from ParamHandler import ParamHandler as PH
from AE import AE
from KNN import KNN
from Metrics import *
#import tensorflow as tf

In [ ]:
ph = PH(file='params/params_2.yaml')
ph.load()
pm = PM(ph)
go = GO(pm, ph)
ppi = PPI(pm, ph, batch_size=200)
ae = AE(pm, ph, ppi, learn_rate=0.0001)

In [ ]:
#ppi.pca()
ae.train(epochs=100)

In [ ]:
c = ae.compress()

In [ ]:
c.shape

In [ ]:
if ph.get('postnormalization', True):
    c = (c[:,0]-c[:,0].mean(axis=0))/np.std(c[:,0], axis=0)

In [ ]:
ae.save()

In [ ]:
knn = KNN(pm, ph, go, c, neighbor_count=10)

In [ ]:
Y = knn.predict()

In [ ]:
met = MetricGrapher(pm, ph, Y, go)
met.new_metric(Metric.Function('posP'), Metric.Function('posR'))
met.new_metric(Metric.Function('negP'), Metric.Function('negR'))
met.new_metric(Metric.Function('tpr'), Metric.Function('fpr'))
met.axies = ['Recall/TPR', 'Precision/TPR']
met.initialize_metrics() 
met.make_graphs(show=True)

In [ ]:
ph.save()